In [1]:
from datasets import Dataset
import pandas as pd
import evaluate
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoTokenizer, set_seed
import os
from sklearn.model_selection import train_test_split
from scipy.special import softmax
import argparse
import logging
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import wandb
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import os
from torch.utils.data import Dataset
import torch
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
from datasets import Dataset
import numpy as np
import numpy as np
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from datasets import Dataset
import pandas as pd
import evaluate
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoTokenizer, set_seed
import os
from sklearn.model_selection import train_test_split
from scipy.special import softmax
import argparse
import logging
import pandas as pd,os
import torch
from statistics import mode
from sklearn.metrics import confusion_matrix
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, Trainer
from datasets import Dataset, load_metric
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
import pickle

c:\Users\Ghiki\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def preprocess_function(examples, **fn_kwargs):
    return fn_kwargs['tokenizer'](examples["text"], truncation=True)

def get_data(train_path, test_path, random_seed):
    """
    function to read dataframe with columns
    """

    train_df = pd.read_json(train_path, lines=True)
    test_df = pd.read_json(test_path, lines=True)
    
    train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['label'], random_state=random_seed)

    return train_df, val_df, test_df

def compute_metrics(eval_pred):

    f1_metric = evaluate.load("f1")

    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    results = {}
    results.update(f1_metric.compute(predictions=predictions, references = labels, average="micro"))

    return results

def fine_tune(train_df, valid_df, checkpoints_path, id2label, label2id, model):

    # pandas dataframe to huggingface Dataset
    train_dataset = Dataset.from_pandas(train_df)
    valid_dataset = Dataset.from_pandas(valid_df)
    
    # get tokenizer and model from huggingface
    tokenizer = AutoTokenizer.from_pretrained(model)     # put your model here
    model = AutoModelForSequenceClassification.from_pretrained(
       model, num_labels=len(label2id), id2label=id2label, label2id=label2id    # put your model here
    )
    
    # tokenize data for train/valid
    tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True, fn_kwargs={'tokenizer': tokenizer})
    tokenized_valid_dataset = valid_dataset.map(preprocess_function, batched=True,  fn_kwargs={'tokenizer': tokenizer})
    

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


    # create Trainer 
    training_args = TrainingArguments(
        output_dir=checkpoints_path,
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_valid_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    # save best model
    best_model_path = checkpoints_path+'/best/'
    
    if not os.path.exists(best_model_path):
        os.makedirs(best_model_path)
    

    trainer.save_model(best_model_path)

def test(test_df, model_path, id2label, label2id):
    
    # load tokenizer from saved model 
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    # load best model
    model = AutoModelForSequenceClassification.from_pretrained(
       model_path, num_labels=len(label2id), id2label=id2label, label2id=label2id
    )
            
    test_dataset = Dataset.from_pandas(test_df)

    tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True,  fn_kwargs={'tokenizer': tokenizer})
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    # create Trainer
    trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )
    # get logits from predictions and evaluate results using classification report
    predictions = trainer.predict(tokenized_test_dataset)
    prob_pred = softmax(predictions.predictions, axis=-1)
    preds = np.argmax(predictions.predictions, axis=-1)
    metric = evaluate.load("bstrai/classification_report")
    results = metric.compute(predictions=preds, references=predictions.label_ids)
    
    # return dictionary of classification report
    return results, preds

In [7]:
random_seed = 0
model = 'roberta-base'
model = 'google/electra-base-discriminator'
model = 'bert-base-uncased'
model = 'distilbert-base-uncased'
model = 'bert-base-multilingual-cased'

df= pd.read_json('datasets/subtaskB_train.jsonl', lines=True)

# df=df.sample(20000)

df.to_json('reducedTrainDataFrame.jsonl', orient='records', lines=True)

train_path =  'reducedTrainDataFrame.jsonl'
test_path =  'datasets/subtaskB_dev.jsonl'

subtask =  'B'
prediction_path = 'reducedPredictedDataFrame.jsonl'

if not os.path.exists(train_path):
    logging.error("File doesnt exists: {}".format(train_path))
    raise ValueError("File doesnt exists: {}".format(train_path))

if not os.path.exists(test_path):
    logging.error("File doesnt exists: {}".format(train_path))
    raise ValueError("File doesnt exists: {}".format(train_path))

if subtask == 'A':
    id2label = {0: "human", 1: "machine"}
    label2id = {"human": 0, "machine": 1}
elif subtask == 'B':
    id2label = {0: 'human', 1: 'chatGPT', 2: 'cohere', 3: 'davinci', 4: 'bloomz', 5: 'dolly'}
    label2id = {'human': 0, 'chatGPT': 1,'cohere': 2, 'davinci': 3, 'bloomz': 4, 'dolly': 5}
else:
    logging.error("Wrong subtask: {}. It should be A or B".format(train_path))
    raise ValueError("Wrong subtask: {}. It should be A or B".format(train_path))

set_seed(random_seed)

train_df, valid_df, test_df = get_data(train_path, test_path, random_seed)

fine_tune(train_df, valid_df, f"{model}/subtask{subtask}/{random_seed}", id2label, label2id, model)

results, predictions = test(test_df, f"{model}/subtask{subtask}/{random_seed}/best/", id2label, label2id)
print(results['macro avg']['f1-score'])

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  5%|▍         | 500/10656 [01:55<38:44,  4.37it/s]

{'loss': 0.8958, 'learning_rate': 1.9061561561561564e-05, 'epoch': 0.14}


  9%|▉         | 1000/10656 [03:49<36:50,  4.37it/s]

{'loss': 0.4968, 'learning_rate': 1.8123123123123126e-05, 'epoch': 0.28}


 14%|█▍        | 1500/10656 [05:43<35:08,  4.34it/s]

...

{'eval_loss': 0.5494930744171143, 'eval_f1': 0.8980712375052795, 'eval_runtime': 95.575, 'eval_samples_per_second': 148.637, 'eval_steps_per_second': 9.291, 'epoch': 3.0}


100%|██████████| 10656/10656 [45:20<00:00,  3.92it/s]


{'train_runtime': 2720.8119, 'train_samples_per_second': 62.652, 'train_steps_per_second': 3.916, 'train_loss': 0.2286621295475029, 'epoch': 3.0}


Map: 100%|██████████| 3000/3000 [00:00<00:00, 5522.50 examples/s]
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100%|██████████| 375/375 [00:16<00:00, 22.18it/s]


0.6076907158380385


In [4]:
results, predictions = test(test_df, f"{model}/subtask{subtask}/{random_seed}/best/", id2label, label2id)
print(results['macro avg']['f1-score'])

Map: 100%|██████████| 3000/3000 [00:00<00:00, 6045.18 examples/s]
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100%|██████████| 375/375 [00:08<00:00, 44.78it/s]


0.5754180918344265


In [5]:
results, predictions

({'0': {'precision': 0.9753086419753086,
   'recall': 0.316,
   'f1-score': 0.4773413897280967,
   'support': 500.0},
  '1': {'precision': 0.8973105134474327,
   'recall': 0.734,
   'f1-score': 0.8074807480748075,
   'support': 500.0},
  '2': {'precision': 0.4484848484848485,
   'recall': 0.592,
   'f1-score': 0.5103448275862069,
   'support': 500.0},
  '3': {'precision': 0.18666666666666668,
   'recall': 0.056,
   'f1-score': 0.08615384615384615,
   'support': 500.0},
  '4': {'precision': 0.9191176470588235,
   'recall': 1.0,
   'f1-score': 0.9578544061302682,
   'support': 500.0},
  '5': {'precision': 0.4493023255813953,
   'recall': 0.966,
   'f1-score': 0.6133333333333333,
   'support': 500.0},
  'accuracy': 0.6106666666666667,
  'macro avg': {'precision': 0.6460317738690792,
   'recall': 0.6106666666666666,
   'f1-score': 0.5754180918344265,
   'support': 3000.0},
  'weighted avg': {'precision': 0.6460317738690792,
   'recall': 0.6106666666666667,
   'f1-score': 0.5754180918344265